# About

These is a base solution of PID.

In [1]:
%matplotlib inline
import pandas
import numpy
import matplotlib.pyplot as plt
import root_numpy
import os, sys

sys.path.append('src')
from data_utils import get_number_particles, generate_data_sample

# Get files https

In [2]:
def get_files_https(files):

    files_http = []

    for one_file in files:

        one_file_path, one_file_tree_name = one_file.split(':')

        one_file_http = one_file_path.replace("/r02/lhcb/jonesc/ANNPID/", "http://www.hep.phy.cam.ac.uk/~jonesc/lhcb/PID/")

        files_http.append((one_file_http, one_file_tree_name))

    return files_http

In [3]:
path_files_txt = '../../../data/configs/training/MC2015Sim09Dev03-TrainingFiles-Mixture-Cambridge.txt'
files = numpy.loadtxt(path_files_txt, dtype='S', delimiter='\n', comments='#')

files_http = get_files_https(files)

print len(files_http)
files_http[:5]

552


[('http://www.hep.phy.cam.ac.uk/~jonesc/lhcb/PID/ProtoParticlePIDtuples/MC/Sim09Devx/Reco15x/Dev03/ANNPID.1.root',
  'ANNPID/DecayTree'),
 ('http://www.hep.phy.cam.ac.uk/~jonesc/lhcb/PID/ProtoParticlePIDtuples/MC/Sim09Devx/Reco15x/Dev03/ANNPID.2.root',
  'ANNPID/DecayTree'),
 ('http://www.hep.phy.cam.ac.uk/~jonesc/lhcb/PID/ProtoParticlePIDtuples/MC/Sim09Devx/Reco15x/Dev03/ANNPID.3.root',
  'ANNPID/DecayTree'),
 ('http://www.hep.phy.cam.ac.uk/~jonesc/lhcb/PID/ProtoParticlePIDtuples/MC/Sim09Devx/Reco15x/Dev03/ANNPID.4.root',
  'ANNPID/DecayTree'),
 ('http://www.hep.phy.cam.ac.uk/~jonesc/lhcb/PID/ProtoParticlePIDtuples/MC/Sim09Devx/Reco15x/Dev03/ANNPID.5.root',
  'ANNPID/DecayTree')]

# Get number of particles

In [5]:
selection = '(!HasMC || MCVertexType==1 || MCVertexType==2) && \
             (TrackLikelihood > -100.0) && \
             (TrackP > 0) && \
             (TrackPt > 0) && (abs(TrackType-3) < 0.1)'

dir_path = '../../../data/MC2015Sim09Dev03/Generation1/Long'
particles_pdg = [11,13,211,321,2212,0]

In [6]:
# log_file_name = dir_path + '/get_number_particles.log'
# numbers_particles_df = get_number_particles(files_http, 
#                                             particles_pdg, 
#                                             selection, 
#                                             log_file_name=log_file_name)

# numbers_particles_df.head()

In [7]:
numbers_particles_df = pandas.read_csv(dir_path + '/number_particles.csv', index_col=0)
#numbers_particles_df.to_csv(dir_path + '/number_particles.csv')
numbers_particles_df.head()

,11,13,211,321,2212,0,http,tree_name
0,1419,23390,285801,42435,32373,60454,http://www.hep.phy.cam.ac.uk/~jonesc/lhcb/PID/...,ANNPID/DecayTree
1,2476,3928,317760,69607,31487,70250,http://www.hep.phy.cam.ac.uk/~jonesc/lhcb/PID/...,ANNPID/DecayTree
2,15424,23188,1584021,241895,166204,359059,http://www.hep.phy.cam.ac.uk/~jonesc/lhcb/PID/...,ANNPID/DecayTree
3,11810,123150,1494633,266713,158086,337539,http://www.hep.phy.cam.ac.uk/~jonesc/lhcb/PID/...,ANNPID/DecayTree
4,12206,128919,1509125,255125,153070,330710,http://www.hep.phy.cam.ac.uk/~jonesc/lhcb/PID/...,ANNPID/DecayTree


# Train generation

In [ ]:
index_sel = numpy.array(numbers_particles_df.index) % 2 == 0
numbers_particles_train = numbers_particles_df[index_sel]

generate_data_sample(  numbers_particles_train, 
                       1000000, 
                       selection, 
                       dir_path + '/data_train.csv', 
                       dir_path + '/get_sample_train.log',
                       dir_path + '/readed_train.log')

In [ ]:
data_train = pandas.read_csv(dir_path + '/data_train.csv', index_col=0, usecols=['MCParticleType', 'TrackType'])
len(data_train)

# Eval generation

In [ ]:
index_sel = numpy.array(numbers_particles_df.index) % 2 == 1
numbers_particles_eval = numbers_particles_df[index_sel]

generate_data_sample(  numbers_particles_eval, 
                       1000000, 
                       selection, 
                       dir_path + '/data_eval.csv', 
                       dir_path + '/get_sample_eval.log',
                       dir_path + '/readed_eval.log')

In [ ]:
data_eval = pandas.read_csv(dir_path + '/data_eval.csv', index_col=0, usecols=['MCParticleType'])
len(data_eval)